<a href="https://colab.research.google.com/github/jamescavanagh/unpackAI/blob/main/07_sizing_and_tta_own_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Computer Vision Sizing and TTA - State-of-the-Art Model

Own refactored code and notes for Chapter 7: *Sizing and TTA* (['07_sizing_and_tta.ipynb'](https://colab.research.google.com/github/vtecftwy/fastbook/blob/master/07_sizing_and_tta.ipynb#scrollTo=6tT8i-1PC9ko))



Rewrite the code for the image classifier that identifies multiple objects in an image using Imagenet; or, use your own data set that is properly labelled and it big enough.
- Only use the code necessary to load the data, create tbe DataLoaders, presize and or augment the images and then train the model using all the knowledge and tools you have learned from the book so far adding the new nowledge you have gained in this chapter.
- In this chapter we are focussing on Mixup and Lable Smoothing (show these features in your notebook)
- Feel free to add any code that you perfer to use and make a note in the code as to why you perferred to use that code
- Add your own comments that can be used for reference in the future.

# Questionaire

### Is using TTA at inference slower or faster than regular inference? Why?

Time tested augmentation is slower because it involves sampling multiple recroppings from the original image. It is a useful techique in certain cases where features could be cropped out by randomized recropping. This can increase the 


### Why does Mixup Prevent the model from being too confident

It blends different images, in order to allow the model to seek to create a match of whatever percent blend that the images used. If it is 100 percent certain, then it is difficult for it in real life



###Imports and Setup

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *

from fastai.vision.all import *



###Preparing the data

In [ ]:

path = untar_data(URLs.IMAGENETTE)

dblock = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = dblock.dataloaders(path, bs=64)

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:1023: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:760.)
  ret = func(*args, **kwargs)


In [ ]:


def get_dls(bs, size):
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       get_items=get_image_files,
                       get_y=parent_label,
                       item_tfms=Resize(460),
                       batch_tfms=[*aug_transforms(size=size, min_scale=0.75),
                                   Normalize.from_stats(*imagenet_stats)])
    return dblock.dataloaders(path, bs=bs)

dls = get_dls(64, 224)

x,y = dls.one_batch()
x.mean(dim=[0,2,3]),x.std(dim=[0,2,3])

(TensorImage([-0.3337, -0.1871, -0.1319], device='cuda:0'),
 TensorImage([1.2113, 1.2294, 1.3043], device='cuda:0'))

In [ ]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.589169,2.406601,0.394324,02:06
1,1.227618,1.181839,0.617998,02:07
2,0.953036,0.887949,0.716206,02:06
3,0.755232,0.755916,0.761016,02:05
4,0.585928,0.562236,0.824122,02:05


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


####*Progressive Resizing*

In [ ]:
dls = get_dls(128, 128)
learn = Learner(dls, xresnet50(n_out=dls.c), loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy)
learn.fit_one_cycle(4, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.604778,3.349528,0.286034,01:54
1,1.224700,1.271022,0.595967,01:54
2,0.948092,0.849190,0.733757,01:53
3,0.739909,0.640563,0.802091,01:54


####*Test Time Augmentation*

In [ ]:
preds,targs = learn.tta()
accuracy(preds, targs).item()

0.8095593452453613

####*Mixup*

In [ ]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), 
                metrics=accuracy, cbs=MixUp())
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.902669,2.532270,0.406273,01:54
1,1.688156,1.850655,0.490665,01:54
2,1.453632,1.016058,0.676624,01:54
3,1.291450,0.779515,0.761016,01:55
4,1.184780,0.694673,0.801344,01:54


####*Lable Smoothing*

In [ ]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=LabelSmoothingCrossEntropy(), 
                metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.808663,2.037147,0.464152,01:54
1,1.543168,1.544442,0.603435,01:54
2,1.306987,1.324062,0.677745,01:54


###Train the Model